# Imports

In [1]:
pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.2 MB/s eta 0:00:00


In [2]:
import os
import cv2
import shutil
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np

# Getting Data

In [3]:
# Origial folder location
data_scr_dir = '/content/drive/MyDrive/Flatiron/Phase 5/Data'


data_dst_folder = '/content/data'


# copy folder from location to local storage

shutil.copytree(data_scr_dir, data_dst_folder)

'/content/data'

In [4]:
# Set up the paths and class labels
data_dir = '/content/data'
class_labels = ['VeryMildDemented', 'ModerateDemented', 'MildDemented', 'NonDemented']
image_size = (28, 28) # Desired image size

In [5]:
# Load the images and labels:
img = []
lbl = []

for label_idx, label in enumerate(class_labels):
    folder_path = os.path.join(data_dir, label)
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file) # get the image
        image = cv2.imread(image_path) # read the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = cv2.resize(image, image_size)  # Resize the image
        img.append(image)
        lbl.append(label_idx)

In [6]:
dataset = np.array(img)
label = np.array(lbl)

In [7]:
#Split into train and test data sets
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)

In [8]:
X_train.shape

(27194, 28, 28, 3)


# Model

In [30]:
from keras.api._v2.keras import activations
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28,3)))

  for i in range(hp.Int('layers', 2, 20)):
    model.add(keras.layers.Dense(
        units=hp.Int('units', min_value=8, max_value=512, step=32),
        activation = hp.Choice('act_', values=['relu'])
        ))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(4, activation='softmax'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [31]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='HyperBand_Dir_v3',
                     project_name='Alzheimer_Image_Tuner')

In [32]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [33]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 51s]
val_accuracy: 0.4642397463321686

Best val_accuracy So Far: 0.7712814807891846
Total elapsed time: 00h 17m 48s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 424 and the optimal learning rate for the optimizer
is 0.0001.



# Train the model

In [34]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
680/680 [==============================] - 7s 6ms/step - loss: 1.2788 - accuracy: 0.4080 - val_loss: 1.0386 - val_accuracy: 0.4970
Epoch 2/50
680/680 [==============================] - 4s 6ms/step - loss: 1.0127 - accuracy: 0.5232 - val_loss: 0.9535 - val_accuracy: 0.5510
Epoch 3/50
680/680 [==============================] - 4s 5ms/step - loss: 0.9481 - accuracy: 0.5549 - val_loss: 0.8727 - val_accuracy: 0.5878
Epoch 4/50
680/680 [==============================] - 4s 6ms/step - loss: 0.8956 - accuracy: 0.5786 - val_loss: 0.8500 - val_accuracy: 0.6019
Epoch 5/50
680/680 [==============================] - 4s 6ms/step - loss: 0.8519 - accuracy: 0.6018 - val_loss: 0.8759 - val_accuracy: 0.5780
Epoch 6/50
680/680 [==============================] - 4s 6ms/step - loss: 0.8069 - accuracy: 0.6280 - val_loss: 0.8545 - val_accuracy: 0.5872
Epoch 7/50
680/680 [==============================] - 4s 5ms/step - loss: 0.7705 - accuracy: 0.6452 - val_loss: 0.7530 - val_accuracy: 0.6520
Epoch 

In [35]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/50
680/680 [==============================] - 8s 5ms/step - loss: 1.2603 - accuracy: 0.4074 - val_loss: 1.0081 - val_accuracy: 0.5082
Epoch 2/50
680/680 [==============================] - 4s 5ms/step - loss: 1.0054 - accuracy: 0.5202 - val_loss: 0.9766 - val_accuracy: 0.5418
Epoch 3/50
680/680 [==============================] - 4s 6ms/step - loss: 0.9416 - accuracy: 0.5549 - val_loss: 0.9198 - val_accuracy: 0.5735
Epoch 4/50
680/680 [==============================] - 4s 5ms/step - loss: 0.8811 - accuracy: 0.5847 - val_loss: 0.9581 - val_accuracy: 0.5628
Epoch 5/50
680/680 [==============================] - 4s 5ms/step - loss: 0.8398 - accuracy: 0.6103 - val_loss: 0.8618 - val_accuracy: 0.5847
Epoch 6/50
680/680 [==============================] - 4s 5ms/step - loss: 0.8025 - accuracy: 0.6325 - val_loss: 0.7784 - val_accuracy: 0.6459
Epoch 7/50
680/680 [==============================] - 4s 6ms/step - loss: 0.7577 - accuracy: 0.6538 - val_loss: 0.7975 - val_accuracy: 0.6249
Epoch 

In [36]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

213/213 [==============================] - 1s 2ms/step - loss: 0.8034 - accuracy: 0.8075
[test loss, test accuracy]: [0.8034020066261292, 0.8074716925621033]


In [37]:

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 2352)              0         
                                                                 
 dense_18 (Dense)            (None, 424)               997672    
                                                                 
 dense_19 (Dense)            (None, 424)               180200    
                                                                 
 dense_20 (Dense)            (None, 424)               180200    
                                                                 
 dense_21 (Dense)            (None, 424)               180200    
                                                                 
 dense_22 (Dense)            (None, 424)               180200    
                                                                 
 dense_23 (Dense)            (None, 424)              